# ABC analysis with SQL 

Hi there! Today we will practice making ABC categorization with SQL.
I'll be using PostgreSQL but SQL synthax will be very generic.

## Let's create a database and import our Superstore file

I've converted `Superstore.xlsx` file to `CSV` format using `Save As` option in my Excel. Make sure to set separator (or it can be called as delimiter) **other than comma** (,) because we have commas in product names and our importing will fail (because of errors in columns count).

I suggest using semicolon (;) as a separator.

To create a database run `pgAdmin`, double click on one of the servers listed and enter password to make connection.


After connecting to server right-click on Databases and select Create Database. Give your database a name **superstore**.

<img src="Screenshots/SQL/create_db.jpg" width="400" />


Now we need to create a table with a structure same as our CSV file.

 There are different tools available to create tables automatically upon importing csv files into database but I don't have them in my disposal.

Instead of that I wrote a small Python code to prepare a SQL command for table creation based on current csv file structure. It analyses columns and data types in csv and make a proper SQL code - just run it and your code will be created. 

In [2]:
import pandas as pd

def get_sql_data_type(dtype, series):
    """Convert pandas data type to SQL data type."""
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATE'
    elif pd.api.types.is_string_dtype(dtype):
        return 'VARCHAR'
    return 'TEXT'

def format_column_name(col_name):
    """Format column names for SQL: convert to lower case and replace spaces with underscores."""
    return col_name.strip().lower().replace(' ', '_')

def generate_sql_create_table(table_name, date_columns=None):
    """
    Generate SQL CREATE TABLE command based on CSV file structure.
    
    :param table_name: Desired name for the SQL table
    :param date_columns: List of column names or indices to treat as dates
    :return: SQL CREATE TABLE command
    """

    # Convert specified columns to datetime
    if date_columns:
        if isinstance(date_columns[0], int):  # If indices are provided
            date_columns = df.columns[date_columns]
        df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%d.%m.%Y', errors='coerce')
    
    # Create SQL column definitions
    columns = []
    for col in df.columns:
        series = df[col]
        formatted_col = format_column_name(col)
        sql_type = get_sql_data_type(series.dtype, series)
        column_definition = f'"{formatted_col}" {sql_type}'
        columns.append(column_definition)
    
    # Construct SQL command
    columns_str = ",\n    ".join(columns)
    sql_command = f"""
CREATE TABLE {table_name} (
    {columns_str}
);
    """
    
    return sql_command

# Load CSV file
df = pd.read_csv('Superstore.csv', sep=';', parse_dates=False)  # Load without parsing dates initially

# Example usage
table_name = 'superstore'   # Desired name for your SQL table

# Specify date columns by name or index
date_columns = ['Order Date', 'Ship Date']  # Can also use indices like [2, 3]

sql_command = generate_sql_create_table(table_name, date_columns=date_columns)
print(sql_command)



CREATE TABLE superstore (
    "row_id" INTEGER,
    "order_id" VARCHAR,
    "order_date" DATE,
    "ship_date" DATE,
    "ship_mode" VARCHAR,
    "customer_id" VARCHAR,
    "customer_name" VARCHAR,
    "segment" VARCHAR,
    "country" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "postal_code" INTEGER,
    "region" VARCHAR,
    "product_id" VARCHAR,
    "category" VARCHAR,
    "sub-category" VARCHAR,
    "product_name" VARCHAR,
    "sales" FLOAT,
    "quantity" INTEGER,
    "discount" FLOAT,
    "profit" FLOAT
);
    


## Creating the table

Right-click on Table and select `Query Tool` - that's a command console where you can use SQL scripts. Paste your CREATE TABLE code that we've created above and press Execute script (F5). Now we've created a table called `superstore`.

<img src='Screenshots/SQL/create_table.jpg' width="400" />

## Importing CSV data

Expand Table view and Right-Click on superstore table name. Select Import|Export Data and Import your CSV file. *Don't forget to select correct delimiter in the Options section*  - **;**

<img src='Screenshots/SQL/import.jpg' width="400" />

# Writing SQL commands for a database

Ok, now we have CSV file loaded into our database and we can start.
In main menu select Query Tool to get to SQL query mode. 



Let's break our task of ABC categorization and ranking into parts:
1) We need to count total sales by each unique product and put them in descending order
2) We need to count cumulative sum of sales
3) We need to calculate percent from total sales for each cumulative sum
4) After making penrentages we can do ABC segmentation
5) Then we can assign ranks to each product
6) And the last - we need to assing these ABC categories and ranks from each product to the main table of orders

Let's create those pieces and make a complete solution from them. We will use CTE (common table expressions) and WINDOW functions

### Total sales by each unique product in descending order

    SELECT
        product_id,
        SUM(sales) AS product_sales
    FROM superstore
    GROUP BY product_id
    ORDER BY SUM(sales) DESC

### Cumulative sum of sales and percentage

    total AS (
    SELECT SUM(product_sales) AS total_sales
    FROM product_sales
    )

    SELECT
        ps.product_id,
        ps.product_sales,
        SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) AS cum_sum,
        (SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) * 1.0 / t.total_sales) AS percent
    FROM product_sales ps
    CROSS JOIN total t

### ABC segmentation and ranking and glueing all together


    WITH product_sales AS (
        SELECT
            product_id,
            SUM(sales) AS product_sales
        FROM superstore
        GROUP BY product_id
        ORDER BY SUM(sales) DESC
    ),
    total AS (
        SELECT SUM(product_sales) AS total_sales
        FROM product_sales
    ),
    abc AS (
        SELECT
            ps.product_id,
            ps.product_sales,
            SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) AS cum_sum,
            (SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) * 1.0 / t.total_sales) AS percent
        FROM product_sales ps
        CROSS JOIN total t
    ),
    ranking AS (
        SELECT 
            product_id,
            product_sales,
            cum_sum,
            percent,
            CASE 
                WHEN percent <= 0.8 THEN 'A'
                WHEN percent <= 0.9 THEN 'B'
                WHEN percent <= 1 THEN 'C'
            END AS category
        FROM abc
    )

    SELECT
        product_id,
        category,
        DENSE_RANK() OVER (PARTITION BY category ORDER BY product_sales DESC) AS rank
    FROM ranking
    ORDER BY category, rank;


### Next we need to modify our main table

To do that we need to create 2 new columns in ``superstore`` table - ``abc_category`` and `sales_rank`.

You need to select `Query Tool` from main menu again to make a new instance of query and run that short piece of code:

    ALTER TABLE superstore
    ADD COLUMN abc_category VARCHAR(1),
    ADD COLUMN sales_rank INT;

### Merging everything

- and now comes the final part - we will merge our categories and rank data with main table based on product id's.

We will use our main SQL query where we've done categorization and ranking as a sub-query. Stay with me ))

    WITH product_sales AS (
        SELECT
            product_id,
            SUM(sales) AS product_sales
        FROM superstore
        GROUP BY product_id
    ),
    total AS (
        SELECT SUM(product_sales) AS total_sales
        FROM product_sales
    ),
    abc AS (
        SELECT
            ps.product_id,
            ps.product_sales,
            SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) AS cum_sum,
            (SUM(ps.product_sales) OVER (ORDER BY ps.product_sales DESC) * 1.0 / t.total_sales) AS percent
        FROM product_sales ps
        CROSS JOIN total t
    ),
    ranking AS (
        SELECT 
            product_id,
            product_sales,
            cum_sum,
            percent,
            CASE 
                WHEN percent <= 0.8 THEN 'A'
                WHEN percent <= 0.9 THEN 'B'
                WHEN percent <= 1 THEN 'C'
            END AS abc_category,
            DENSE_RANK() OVER(PARTITION BY 
            CASE 
                WHEN percent <= 0.8 THEN 'A'
                WHEN percent <= 0.9 THEN 'B'
                WHEN percent <= 1 THEN 'C'
            END ORDER BY product_sales DESC) AS sales_rank
        FROM abc
    )

    UPDATE superstore s
    SET abc_category = r.abc_category,
        sales_rank = r.sales_rank
    FROM ranking r
    WHERE s.product_id = r.product_id;


# Finish line

ok, if everything is done right we can take a look at our table and see 2 new columns with ABC category and sales rank for each product.

    SELECT * FROM superstore LIMIT 10

<img src='Screenshots/SQL/final.jpg' width="800" />

Have a nice day and see you at ABC analysis with Tableau next time!